<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/train_galactic_comm_ai_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
train_galactic_comm_ai.py

Quantum‐informed AI pipeline for GalacticCommAI:

1. Synthetic dataset of 5 channel parameters → 3 comm metrics
2. Physics‐informed residual enforcing toy quantum link laws
3. MLP with LayerNorm & Dropout for robustness
4. MC‐Dropout inference for uncertainty quantification
5. Training loop with AdamW, ReduceLROnPlateau, gradient clipping, early stopping
6. Visualizations: loss curves, scatter plots, uncertainty heatmap
"""

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt

# ------------------------------------------------------------------------------
# 1. Synthetic Quantum Channel Dataset
# ------------------------------------------------------------------------------
class GalacticCommDataset(Dataset):
    def __init__(self, n_samples=6000, seed=42):
        np.random.seed(seed)
        # Inputs: entanglement fidelity E, decoherence rate D,
        # noise level N, link distance L, repeater spacing S
        E = np.random.uniform(0.7, 1.0,   (n_samples,1)).astype(np.float32)
        D = np.random.uniform(1e-3, 1e-1, (n_samples,1)).astype(np.float32)
        N = np.random.uniform(0.0, 0.05,  (n_samples,1)).astype(np.float32)
        L = np.random.uniform(1e3, 1e6,   (n_samples,1)).astype(np.float32)
        S = np.random.uniform(1e2, 1e4,   (n_samples,1)).astype(np.float32)

        X_raw = np.hstack([E, D, N, L, S])

        # Toy physics laws for targets:
        # signal_clarity ≈ E * exp(-D * L / 1e6) - N
        sc = E * np.exp(-D * L / 1e6) - N

        # quantum_stability ≈ E / (1 + D + N)
        qs = E / (1.0 + D + N)

        # error_rate ≈ N + D * (L / 1e6)
        er = N + D * (L / 1e6)

        Y_raw = np.hstack([sc, qs, er]).astype(np.float32)
        # add small noise
        Y_raw += 0.01 * Y_raw.std(axis=0) * np.random.randn(*Y_raw.shape).astype(np.float32)

        # compute normalization stats
        self.X_mean, self.X_std = X_raw.mean(0), X_raw.std(0) + 1e-6
        self.Y_mean, self.Y_std = Y_raw.mean(0), Y_raw.std(0) + 1e-6

        # standardize
        self.X = (X_raw - self.X_mean) / self.X_std
        self.Y = (Y_raw - self.Y_mean) / self.Y_std

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = torch.from_numpy(self.X[idx])
        y = torch.from_numpy(self.Y[idx])
        return x, y

# ------------------------------------------------------------------------------
# 2. Model Definition
# ------------------------------------------------------------------------------
class GalacticCommAI(nn.Module):
    def __init__(self, input_dim=5, hidden_dims=(64,64), output_dim=3, p_drop=0.1):
        super().__init__()
        layers, d = [], input_dim
        for h in hidden_dims:
            layers += [
                nn.Linear(d, h),
                nn.LayerNorm(h),
                nn.ReLU(),
                nn.Dropout(p_drop)
            ]
            d = h
        layers.append(nn.Linear(d, output_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

# ------------------------------------------------------------------------------
# 3. Physics‐Informed Residual Loss
# ------------------------------------------------------------------------------
def physics_residual(pred, X, stats):
    # denormalize inputs & preds
    X_den = X * stats['X_std'] + stats['X_mean']
    E, D, N, L, S = X_den.t()
    Y_den = pred * stats['Y_std'] + stats['Y_mean']
    sc_p, qs_p, er_p = Y_den.t()

    # true toy formulas
    sc_t = E * torch.exp(-D * L / 1e6) - N
    qs_t = E / (1.0 + D + N)
    er_t = N + D * (L / 1e6)

    loss_sc = nn.MSELoss()(sc_p, sc_t)
    loss_qs = nn.MSELoss()(qs_p, qs_t)
    loss_er = nn.MSELoss()(er_p, er_t)

    return loss_sc + loss_qs + loss_er

def total_loss(pred, true, X, stats, lam=1.0):
    mse  = nn.MSELoss()(pred, true)
    phys = physics_residual(pred, X, stats)
    return mse + lam * phys, mse, phys

# ------------------------------------------------------------------------------
# 4. MC‐Dropout Inference
# ------------------------------------------------------------------------------
def mc_dropout_predict(model, x, T=50):
    model.train()
    preds = []
    with torch.no_grad():
        for _ in range(T):
            preds.append(model(x))
    arr = torch.stack(preds, 0)
    return arr.mean(0), arr.std(0)

# ------------------------------------------------------------------------------
# 5. Training Loop
# ------------------------------------------------------------------------------
def train(model, train_loader, val_loader, stats, device,
          lr=1e-3, wd=1e-5, lam=1.0, epochs=100, patience=10):
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                     mode='min',
                                                     factor=0.5,
                                                     patience=5)

    best_val, wait = float('inf'), 0
    history = {'train': [], 'val': []}

    for epoch in range(1, epochs + 1):
        model.train()
        train_loss = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            loss, _, _ = total_loss(pred, yb, xb, stats, lam)
            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            train_loss += loss.item() * xb.size(0)
        train_loss /= len(train_loader.dataset)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                pred = model(xb)
                loss, _, _ = total_loss(pred, yb, xb, stats, lam)
                val_loss += loss.item() * xb.size(0)
        val_loss /= len(val_loader.dataset)

        scheduler.step(val_loss)
        history['train'].append(train_loss)
        history['val'].append(val_loss)
        print(f"Epoch {epoch:03d} | Train {train_loss:.4e} | Val {val_loss:.4e}")

        if val_loss + 1e-8 < best_val:
            best_val, wait = val_loss, 0
            torch.save(model.state_dict(), "best_galactic_comm.pth")
        else:
            wait += 1
            if wait >= patience:
                print(f"Early stopping at epoch {epoch}")
                break

    model.load_state_dict(torch.load("best_galactic_comm.pth"))
    return history

# ------------------------------------------------------------------------------
# 6. Visualization Helpers
# ------------------------------------------------------------------------------
def plot_history(hist):
    plt.figure()
    plt.plot(hist['train'], label='Train')
    plt.plot(hist['val'],   label='Val')
    plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.legend(); plt.show()

def plot_scatter(y_true, y_pred, title):
    plt.figure()
    plt.scatter(y_true, y_pred, s=5, alpha=0.6)
    m, M = y_true.min(), y_true.max()
    plt.plot([m, M], [m, M], 'r--')
    plt.title(title); plt.xlabel("True"); plt.ylabel("Pred"); plt.show()

def plot_uncertainty(model, stats, device):
    # vary E vs D, fix N,L,S at mean
    Es = np.linspace(0.7, 1.0, 100, dtype=np.float32)
    Ds = np.linspace(1e-3, 1e-1, 100, dtype=np.float32)
    Eg, Dg = np.meshgrid(Es, Ds)

    pts = Eg.size
    grid = torch.zeros((pts, 5), device=device)
    grid[:,0] = torch.from_numpy(Eg.ravel()).to(device)
    grid[:,1] = torch.from_numpy(Dg.ravel()).to(device)
    for i in (2,3,4):
        grid[:,i] = stats['X_mean'][i]

    Xn = (grid - stats['X_mean']) / stats['X_std']
    _, std = mc_dropout_predict(model, Xn, T=100)
    std_map = std[:,0].cpu().numpy().reshape(Eg.shape)

    plt.figure(figsize=(6,5))
    plt.pcolormesh(Eg, Dg, std_map, cmap='inferno', shading='auto')
    plt.colorbar(label='Std Signal Clarity')
    plt.xlabel('Entanglement Fidelity E'); plt.ylabel('Decoherence Rate D')
    plt.title('Uncertainty Heatmap: Signal Clarity')
    plt.show()

# ------------------------------------------------------------------------------
# 7. Main Execution
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    ds = GalacticCommDataset(n_samples=6000)
    stats = {
        'X_mean': torch.tensor(ds.X_mean, device=device),
        'X_std' : torch.tensor(ds.X_std,  device=device),
        'Y_mean': torch.tensor(ds.Y_mean, device=device),
        'Y_std' : torch.tensor(ds.Y_std,  device=device),
    }

    n_val = int(0.2 * len(ds))
    tr, va = random_split(ds, [len(ds) - n_val, n_val])
    tr_ld = DataLoader(tr, batch_size=128, shuffle=True)
    va_ld = DataLoader(va, batch_size=256)

    model = GalacticCommAI().to(device)
    history = train(model, tr_ld, va_ld, stats, device)

    plot_history(history)

    X_all = torch.from_numpy(ds.X).float().to(device)
    with torch.no_grad():
        Yp_norm = model(X_all).cpu().numpy()
    Yp = Yp_norm * ds.Y_std + ds.Y_mean
    Yt = ds.Y
    for i, name in enumerate(['Signal Clarity','Quantum Stability','Error Rate']):
        plot_scatter(Yt[:,i], Yp[:,i], name)

    plot_uncertainty(model, stats, device)